#Deep Learning for QSAR and Cell Identification
_by David Holmberg (February 2023), adapted from materials prepared by Phil Harrison (2021)_

### Artificial neural networks for QSAR

#### Dataset
For this exercise we will use the same dataset of aqueous solubility of 1142 diverse chemical compounds as you previously explored during the QSAR lab last week. However, here we will only use the PhysChem descriptors.

#### Modelling comparisons
1. Compare the results of linear regression to those of a simple neural network with no hidden layers and no non-linear activation functions
2. Compare the results of a simple neural network to those of a deeper neural network with hidden layers and non-linear .

#### Aims
* to see the link between neural networks and linear regression
* to learn the basics of how to define, compile, fit and evaluate neural networks via TensorFlow.

#### Note
We will be using the open-source machine learning framework TensorFlow (https://www.tensorflow.org) and Keras (https://keras.io) for our neural networks. TensorFlow was developed by the Google Brain team and is a widely used machine learning frameworks in research and industry and Keras was the most popular higher-level API that runs atop TensorFlow. Today Keras has been entirely absorbed by Tensorflow, making it signidicantly easier to handle without touching the boilerplate.

## Load packages

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, datasets
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Helper libraries
import random
import cv2
import re
import glob
import os
import random
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from datetime import datetime

from tifffile import imread
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
print(tf.__version__)


## Plotting functions

In [ ]:
def plot_history(model_history, model_name):
    fig = plt.figure(figsize=(15,5), facecolor='w')
    ax = fig.add_subplot(121)
    ax.plot(model_history.history['loss'])
    ax.plot(model_history.history['val_loss'])
    ax.set(title=model_name + ': Model loss', ylabel='Loss', xlabel='Epoch')
    ax.legend(['Train', 'Test'], loc='upper right')
    ax = fig.add_subplot(122)
    ax.plot(np.log(model_history.history['loss']))
    ax.plot(np.log(model_history.history['val_loss']))
    ax.set(title=model_name + ': Log model loss', ylabel='Log loss', xlabel='Epoch')
    ax.legend(['Train', 'Test'], loc='upper right')
    plt.show()
    plt.close()

## Load and preprocess data

#### Load and check shape of X and y

In [ ]:
X = np.load('data/X_qsar.npy')
y = np.load('data/y_qsar.npy')
print(X.shape)
print(y.shape)

#### Split into training and test sets and standardize the data
Here we will just have a training and test set, so our results will not be as rigerous as those you got with cross-validation in the supervised machine learning lab.

In [ ]:
n_train = int(len(y) * 0.7) # 70% of data for training and 30% for testing

random.seed(1234)
indices = np.arange(len(y))
random.shuffle(indices)

# X_train0 is our training data prior to standardization
X_train0, X_test0 = X[indices[:n_train]], X[indices[n_train:]]
y_train, y_test = y[indices[:n_train]], y[indices[n_train:]]

# standardize X_train0 and X_test0 to give X_train and X_test
scaler = StandardScaler().fit(X_train0)
X_train = scaler.transform(X_train0)
X_test = scaler.transform(X_test0)

## Linear Regression

In [ ]:
LR_model = LinearRegression()
LR_model.fit(X_train, y_train)
LR_pred = LR_model.predict(X_test)
LR_mse = mean_squared_error(y_test, LR_pred)
print('Linear Regression: MSE = ' + str(np.round(LR_mse, 3)))

## Artifical neural network as a linear regression
If we define a neural network with no hidden layers and no non-linear activations we essentailly get the same results as we do with basic linear regression. The results below should help clarify that to you (there are some minor differences hovever, hence the MSE for the neural network will not be _exactly_ the same as the results above for linear regression, but they are neverthelss very close).

<p>
    <img src="figs/lin-reg.png" alt="drawing" style="width:1200px;"/>
    <center>Figure 1. Our neural network version of linear regression.</center>
</p>

#### Define and summarise the model

In [ ]:
inps = layers.Input(shape=X_train[0].shape)
preds = layers.Dense(1)(inps)

ANN1 = models.Model(inputs=inps, outputs=preds)
ANN1.summary()

#### Compile and fit the model
The learning rate and optimizer chosen below are both things that can be changed when one explores hyper parameter options, different architectures and what not. Below we use a learning rate (lr) of 0.001 (a common default learning rate) and the 'RMSprop' optimizer.

In [ ]:
lr = 0.001

ANN1.compile(optimizer=optimizers.RMSprop(learning_rate=lr), loss='mse')

In [ ]:
history = ANN1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, verbose=0)
plot_history(history, 'ANN1')

#### Evaluate the model

In [ ]:
ANN1_mse = ANN1.evaluate(X_test, y_test, verbose=0)
print('ANN1: MSE = ' + str(np.round(ANN1_mse, 3)))

## Going deeper with ANNs 
In the cells below we define, compile, fit and evaluate a neural network model with:
* two hiiden layers, each with 32 neurons and non-linear activations (relu)
* a dropout layer at the end with a dropout rate of 0.2

<p>
    <img src="figs/relu-activation.png" alt="drawing" style="width:500px;"/>
    <center>Figure 2. relu activation.</center>
</p>

Dropout can help to avoid overfitting, much as L1 and L2 regularizations do (as you explored in the supervise machine learning lab). In the model loss plots (below) this stops the test loss from increasing as you train for more epochs.

Some quotes from a paper I co-authored called "Deep Learning in Image Cytometry: A Review" (https://onlinelibrary.wiley.com/doi/full/10.1002/cyto.a.23701):

"_Overfitting occurs when the parameters of a model fit too closely to the input training data, without capturing the underlying distribution, and thus reducing the model’s ability to generalize to other datasets_".

DROPOUT: "_A regularization technique that reduces the interdependent learning among the neurons to prevent overfitting. Some neurons are randomly “dropped,” or disconnected from other neurons, at every training iteration, removing their influence on the optimization of the other neurons. Dropout creates a sparse network composed of several networks—each trained with a subset of the neurons. This transformation into an ensemble of networks hugely decreases the possibility of overfitting, and can lead to better generalization and increased accuracy_".

<p>
    <img src="figs/dropout.png" alt="drawing" style="width:1200px;"/>
    <center>Figure 3. Dropout.</center>
</p>

In what comes below there are no missing cells or code lines for you to fill in, this is simply an example. But pay attention to how the code is written below as in the jupyter notebook 'day1_part2' there will be missing parts that you will have to fill in. The code below will help you with those later tasks.

In [ ]:
inps = layers.Input(shape=X_train[0].shape)
x = layers.Dense(32, activation='relu')(inps)
x = layers.Dense(32, activation='relu')(x)
### TODO add another layer to the network here:

x = layers.Dropout(0.2)(x)
preds = layers.Dense(1)(x)

ANN2 = models.Model(inputs=inps, outputs=preds)
ANN2.summary()

In [ ]:
lr = 0.001

# TODO compile your model

In [ ]:
# TODO train your model

In [ ]:
ANN2_mse = ANN2.evaluate(X_test, y_test, verbose=0)
print('ANN2: MSE = ' + str(np.round(ANN2_mse, 3)))

above you should have gotten the lowest MSE for the neural network, followed by the random forest and then the support vector regressor. These results are however not the final say. Different hyper parameter settings for any of these machine learning algorithms could change the rankings. For neural networks there are many hyper parameters that one could explore, including the network architecture, the number of layers, the number of neurons per layer, the drop out rate(s), the learning rate and the optimizer to use. In later labs this week we will explore these, and additional, choices. A full comparison would also better be done via cross validation as our results above are also affected by the train/test splitting of the data...

### CNN for Image-Based Cell Classification

### Dataset
We will be using a simplified subset of the data analysed in the paper "Deep Convolutional Neural Networks For Detecting Cellular Changes Due To
Malignancy" by Wieslander et al. (2017) (http://openaccess.thecvf.com/content_ICCV_2017_workshops/papers/w1/Wieslander_Deep_Convolutional_Neural_ICCV_2017_paper.pdf). The CNNs we'll be training in this exercise will be more basic than those used in that paper.


<p>
    <img src="figs/oral_cancer_summary.png" alt="drawing"  style="width:800px;"/>
    <center>Figure 1. Oral cancer data summary.</center>
</p>

Example cell images Weislander et al. used are shown below. These images were greyscale and 80 x 80 pixels.

<p>
    <img src="figs/cancer_cells.png" alt="drawing" style="width:800px;"/>
    <center>Figure 2. Example oral cancer cell images.</center>
</p>

To simplify the data for the exercise the images have been cropped into 48x48 pixels around the cell. All the images were then shuffled and allocated to the training, validation or test set. Hence, here we are solving a simpler classification task than the more robust one carried out by Weislander et al. where the data were not shuffled and cells from a given patient could only be in one of the three sets. A more robust and generalizable approach would be to do what Weislander et al. did, so keep this in mind.

### Note
Some of the code cells below are boiler plate code, for getting things in the right format, plotting various things and what not. You are encouraged to read and understand what the cells do, it is not required to complete the exercise. Other cells contain either minimal or partial code that you will need to implement. Regardless you need to run all cells below for the notebook to function.

Also below we use `generators` to load the data from directories in small batches. This is useful if your dataset is bigger than fits into the GPU Memory (VRAM). GPUs are super fast for fitting neural networks, much faster than CPUs, but have limited VRAM, and limited access to system RAM. There are other ways of handling data loading, which will be useful should you decide to use deep-learning in the future.

In [ ]:
#Plotting and Summarizing functions
def plot_history(model_history, model_name):
    fig = plt.figure(figsize=(20,6), facecolor='w')
    ax = fig.add_subplot(121)
    ax.plot(model_history.history['loss'])
    ax.plot(model_history.history['val_loss'])
    ax.set(title=model_name + ': Model loss', ylabel='Loss', xlabel='Epoch')
    ax.legend(['Training', 'Validation'], loc='upper left')
    ax = fig.add_subplot(122)
    ax.plot(model_history.history['accuracy'])
    ax.plot(model_history.history['val_accuracy'])
    ax.set(title=model_name + ': Model Accuracy', 
           ylabel='Accuracy', xlabel='Epoch')
    ax.legend(['Training', 'Validation'], loc='upper left')
    plt.show()
    plt.close()

def plot_confusion_matrix(cm, classes, model_name,
                          cmap=plt.cm.Blues):
    title = model_name
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

def plot_sample_images():
    # plot 5 healthy cells form training data
    print('random sample of healthy cells from training set')
    print('')
    all_cells = glob.glob(train_healthy_dir + '/' '*')
    n_cells = len(all_cells)
    to_plot = np.random.choice(n_cells, 5, replace=False)

    plt.figure(figsize=(20, 120), facecolor='w')

    for i in range(5):
        im = imread(all_cells[to_plot[i]])
        sub_index = 151 + i
        plt.subplot(sub_index)
        plt.imshow(im[:, :, 0], cmap='gray')
        plt.xticks([])
        plt.yticks([])


    plt.show()
    plt.close()

    # plot 5 tumor cells form training data
    print('')
    print('random sample of tumor cells from training set')
    print('')
    all_cells = glob.glob(train_tumor_dir + '/' '*')
    n_cells = len(all_cells)
    to_plot = np.random.choice(n_cells, 5, replace=False)

    plt.figure(figsize=(20, 120), facecolor='w')

    for i in range(5):
        im = imread(all_cells[to_plot[i]])
        sub_index = 151 + i
        plt.subplot(sub_index)
        plt.imshow(im[:, :, 0], cmap='gray')
        plt.xticks([])
        plt.yticks([])


    plt.show()
    plt.close()

def data_summary():
    print('no. training healthy images:', len(os.listdir(train_healthy_dir)))
    print('no. training tumor images:', len(os.listdir(train_tumor_dir)))
    print('')
    print('no. validation healthy images:', len(os.listdir(validation_healthy_dir)))
    print('no. validation tumor images:', len(os.listdir(validation_tumor_dir)))
    print('')
    print('no. test healthy images:', len(os.listdir(test_healthy_dir)))
    print('no. test tumor images:', len(os.listdir(test_tumor_dir)))

In [ ]:
# directory we will store our dataset
base_dir = 'HPV/data_v3'

# directories for our training, validation and test splits
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# directory with our training healthy cell images
train_healthy_dir = os.path.join(train_dir, 'healthy')

# directory with our training tumor cell images
train_tumor_dir = os.path.join(train_dir, 'tumor')

# directory with our validation healthy cell images
validation_healthy_dir = os.path.join(validation_dir, 'healthy')

# directory with our validation tumor cell images
validation_tumor_dir = os.path.join(validation_dir, 'tumor')

# directory with our test healthy cell images
test_healthy_dir = os.path.join(test_dir, 'healthy')

# directory with our test tumor cell images
test_tumor_dir = os.path.join(test_dir, 'tumor')

In [ ]:
data_summary()

In [ ]:
plot_sample_images()

In [ ]:
x_len = 48
y_len = 48
batch_size = 32
n_epochs = 50

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

print('TRAINING DATA:')
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(x_len, y_len),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary')

print('')
print('VALIDATION DATA:')
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(x_len, y_len),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary',
    shuffle=False)

print('')
print('TEST DATA:')
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(x_len, y_len),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary',
    shuffle=False)

train_steps = (len(os.listdir(train_healthy_dir)) + len(os.listdir(train_tumor_dir))) // batch_size
validation_steps = (len(os.listdir(validation_healthy_dir)) + len(os.listdir(validation_tumor_dir))) // batch_size
test_steps = (len(os.listdir(test_healthy_dir)) + len(os.listdir(test_tumor_dir))) // batch_size
    
def valid_evaluate(model):
    y_pred = model.predict_generator(validation_generator, validation_steps+1)
    y_pred = y_pred > 0.5
    y_true = validation_generator.classes
    class_names = ['healthy', 'tumor']
    
    cnf_matrix = confusion_matrix(y_true, y_pred)
    np.set_printoptions(precision=2)
    plt.figure(figsize=(15,5), facecolor='w')
    plot_confusion_matrix(cnf_matrix, classes=class_names, model_name='confusion matrix for validation data')
    plt.show()
    plt.close()
    
    print('')
    print('classification report for validation data:')
    print(classification_report(y_true, y_pred, digits=3))

def test_evaluate(model):
    y_pred = model.predict_generator(test_generator, test_steps+1)
    y_pred = y_pred > 0.5
    y_true = test_generator.classes
    class_names = ['healthy', 'tumor']
    
    cnf_matrix = confusion_matrix(y_true, y_pred)
    np.set_printoptions(precision=2)
    plt.figure(figsize=(15,5), facecolor='w')
    plot_confusion_matrix(cnf_matrix, classes=class_names, model_name='confusion matrix for test data')
    plt.show()
    plt.close()
    
    print('')
    print('classification report for test data:')
    print(classification_report(y_true, y_pred, digits=3))

#### Basic convolutional neural network (CNN)

Convolutional Neural Networks (CNNs) use convolutions instead of the normal fully connected layers, which have proven to be highly successful for 2D-data tasks. By convolving filters on the input layer and outputting the results to the next layer, the CNN "detects" (or learns) features at different levels of abstraction throughout the network. With lower-level abstractions (like edges and blobs) in the early layers, and higher-level abstractions (like cells) in deeper layers. The figure below shows the LeNet inspired CNN that we will shortly be implementing.

<p>
    <img src="figs/CNN.png" alt="drawing" style="width:1200px;"/>
</p>

#### Example CNN
The following cells briefly show how to define, compile, and train a model:

In [ ]:
in_data = layers.Input((x_len, y_len, 1)) # input image dimensions

# convolution and pooling layers
x = layers.Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu')(in_data)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# fully connected layers
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
out = layers.Dense(1, activation='sigmoid')(x)

CNN1 = models.Model(inputs=in_data, outputs=out)
CNN1.summary()

In [ ]:
CNN1.compile(loss='binary_crossentropy',
            optimizer=optimizers.Adam(learning_rate=0.001),
            metrics=['accuracy'])

In [ ]:
history = CNN1.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=n_epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    verbose=2)

plot_history(history, 'CNN1')

valid_evaluate(CNN1)

## your own architectures
In the code cells below try some alternative model architectures. Use the 'weighted avg f1-score' on the validation data to compare different model architectures and to decide which one was best.

* try fitting two different models with different numbers of convolutional layers (you can have between two and five for this data, more than five convolutional layers does not work for this data due to the shrinking of the spatial dimensions of the filter maps as they go through max pooling)
* you can also play around with the number of filters for each convolutional layer
* and you can experiment with different values for the dropout rate, or with no dropout at all (by either setting the dropout rate to 0.0 or removing the dropout code line all together)
* finally, you are free to explore different numbers of neurons for the final dense layer (just before our prediction (preds) layer).

#### Notes: 
* if your model does not seem to have converged after 25 epochs (the default value) you can also raise the number of epochs to train for.
* name your model CNN2, CNN3 etc.

In [ ]:
# TODO Define you own CNN here

In [ ]:
# TODO Compile your CNN here

In [ ]:
# TODO Train and Evaluate your CNN here

#### Adding data augmentation

Data augmentation is a way to try improving the performance of a model by artificially inflating the amount of annotated training data. A common approach of doing this is by either mirroring or rotating the data. 

We will use the same 8x data augmentation as used by Wieslander et al.:

<p>
    <img src="figs/augmentation.png" alt="drawing" style="width:1000px;"/>
</p>

In [ ]:
# we add the data augmentations above via training data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(x_len, y_len),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary')

In [ ]:
# TODO Copy your CNN here and rename it to something else. for example, CNN_aug

In [ ]:
# TODO Compile your new model

In [ ]:
# TODO Train you new model

#### Regularization
Besides using dropout we can also explore L1 and/or L2 regularization. To add regularization to you model you add ```kernel_regularizer=l2(0.0001)``` to your layer definitions.
i.e you turn

```x = layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')(x)```

into 

```x = layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.0001))(x)```

In [ ]:
# TODO define your new model, name it something new. I.e CNN_L2. You are free to name it something else, including CNN_L1 if you feel like using l1 regularisation instead:


In [ ]:
# TODO Compile your model

In [ ]:
# TODO Train your model

#### Final Evaluation of the models

You should now evaluate the innitial example model and the three models you've defined. The code for evaluating has been provided, and you only need to call on it using the correct funtion name and function inputs.

In [ ]:
#Here you will use the provided code to evaluate the innitial model>
test_evaluate(CNN1)


In [ ]:
#Evaluate your own model


In [ ]:
#Evaluate your own model with augmentation

In [ ]:
#Evaluate your own model with regularization

#### Final Notes

You've now finished the lab. Due to the stochastic nature of how Deep Learning works, you have likely all gotten different results. This is expected.